In [1]:
import os
import glob
import re
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np

In [2]:
def get_log(folder_path):
    log_files = glob.glob(os.path.join(folder_path, '*.log'))
    if not log_files:
        return None
    latest_file = max(log_files, key=os.path.getmtime)
    return latest_file

def read_log(log_path):
    with open(log_path) as f:
        content=f.readlines()[::-1]
        for line in content:
            if " - Average:" in line:
                return line[31:-1]

In [3]:
names=["STGCN","GWNET","ASTGCN","AGCRN","STGODE","STTN","DCRNN","DSTAGNN","LSTM","TrustEnergy"]
metrics=["MAE","RMSE","MAPE","MPIW","WINK","COV"]
datasets=["NYISO","CAISO"]
cat_type = CategoricalDtype(categories=names, ordered=True)

In [4]:
names=["ARIMA","SARIMA"]
metrics=["MSE","MAE"]
datasets=["sz_taxi_od","sz_bike_od","sz_subway_od",] #
cat_type = CategoricalDtype(categories=names, ordered=True)

In [5]:
names=["HA_OD","HL_OD","ARIMA","SARIMA","LSTM_OD","GMEL","GWNET_OD","STGCN_OD","HMDLF","MPGCN_OD","STZINB","STTN","AGCRN_OD","ASTGCN_OD","STGODE_OD","ODMixer"]
metrics=["MSE","MAE"]
datasets=["sz_taxi_od","sz_bike_od","sz_subway_od",] #
cat_type = CategoricalDtype(categories=names, ordered=True)

In [6]:
path="/home/dy23a.fsu/st/result"
rows = []
for name in names:
    for dataset in datasets:
        path_=f"{path}/{name}/{dataset}"
        if log:=get_log(path_):
            res=read_log(log)
            if res is None:
                continue
            row = {'Dataset': dataset, 'Model': name}
            m = dict(re.findall(r'(\w+): ([\-\d\.]+)', res))
            not_keys=[i for i in m.keys() if i not in metrics]
            for i in not_keys:
                del m[i]
            
            row.update(m)
            rows.append(row)


df = pd.DataFrame(rows)
df['Model'] = df['Model'].astype(cat_type)
df_sorted = df.sort_values(by=['Dataset','Model',])
print(df_sorted)


         Dataset      Model     MSE    MAE
1     sz_bike_od      HA_OD   3.653  0.068
4     sz_bike_od      HL_OD   2.788  0.156
7     sz_bike_od      ARIMA   3.199  0.071
10    sz_bike_od    LSTM_OD   6.139  0.176
13    sz_bike_od       GMEL   6.336  0.279
16    sz_bike_od   GWNET_OD   0.784  0.132
19    sz_bike_od   STGCN_OD   3.053  0.184
22    sz_bike_od      HMDLF   6.561  0.217
25    sz_bike_od     STZINB   0.885  0.129
28    sz_bike_od   AGCRN_OD   1.095  0.180
31    sz_bike_od  ASTGCN_OD   1.502  0.188
34    sz_bike_od  STGODE_OD   2.608  0.219
37    sz_bike_od    ODMixer   3.476  0.314
2   sz_subway_od      HA_OD  10.488  0.138
5   sz_subway_od      HL_OD   9.330  0.244
8   sz_subway_od      ARIMA  10.522  0.134
11  sz_subway_od    LSTM_OD  10.188  0.215
14  sz_subway_od       GMEL  10.590  0.259
17  sz_subway_od   GWNET_OD   1.382  0.080
20  sz_subway_od   STGCN_OD   6.545  0.210
23  sz_subway_od      HMDLF  10.897  0.245
26  sz_subway_od     STZINB   2.123  0.158
29  sz_subw

In [7]:
df_sorted["Dataset"] = df_sorted["Dataset"].str.replace("sz_", "").str.replace("_od", "")  # 只保留 bike/taxi/subway
df_sorted[["MSE", "MAE"]] = df_sorted[["MSE", "MAE"]].apply(pd.to_numeric)
df_pivot = df_sorted.pivot_table(
    index="Model",
    columns="Dataset",
    values=["MSE", "MAE"],
    observed=False
)
df_pivot.columns = [f"{ds}_{metric}" for metric, ds in df_pivot.columns]
df_pivot = df_pivot.reset_index()
col_order = ["Model","taxi_MSE", "taxi_MAE", "bike_MSE", "bike_MAE", "subway_MSE", "subway_MAE"]
df_pivot = df_pivot[col_order]
print(df_pivot)

        Model  taxi_MSE  taxi_MAE  bike_MSE  bike_MAE  subway_MSE  subway_MAE
0       HA_OD     0.519     0.099     3.653     0.068      10.488       0.138
1       HL_OD     0.399     0.116     2.788     0.156       9.330       0.244
2       ARIMA     0.536     0.099     3.199     0.071      10.522       0.134
3     LSTM_OD     1.022     0.181     6.139     0.176      10.188       0.215
4        GMEL     0.839     0.209     6.336     0.279      10.590       0.259
5    GWNET_OD     0.237     0.097     0.784     0.132       1.382       0.080
6    STGCN_OD     0.298     0.111     3.053     0.184       6.545       0.210
7       HMDLF     1.442     0.225     6.561     0.217      10.897       0.245
8      STZINB     0.259     0.120     0.885     0.129       2.123       0.158
9    AGCRN_OD     0.264     0.113     1.095     0.180       1.325       0.099
10  ASTGCN_OD     0.317     0.114     1.502     0.188       2.698       0.127
11  STGODE_OD     0.334     0.126     2.608     0.219       7.20